<a href="https://colab.research.google.com/github/tristan-quant/real-estate-simulation/blob/main/profit_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy-financial
import numpy as np
import numpy_financial as npf
import pandas as pd

# ===== 입력값 =====
매입가 = 320_000_000  # 엔 단위
LTV목록 = [0.6, 0.7]
CapRate목록 = [0.03, 0.04]
대출이자율 = 0.02
대출기간 = 25
보유기간 = 10
법인세율 = 0.3
NOI성장률 = 0.03
ExitCapRate조정 = -0.005  # 매입 CapRate보다 낮게 설정

# ===== 결과 저장용 리스트 =====
결과 = []

for LTV in LTV목록:
    for CapRate in CapRate목록:
        대출금 = 매입가 * LTV
        자기자본 = 매입가 - 대출금
        NOI = 매입가 * CapRate
        ExitCapRate = CapRate + ExitCapRate조정

        # 원리금 균등상환
        r = 대출이자율
        n = 대출기간
        연간상환액 = 대출금 * (r * (1 + r)**n) / ((1 + r)**n - 1)
        대출잔액 = 대출금

        현금흐름 = [-자기자본]
        세후이익목록 = []

        for year in range(1, 보유기간+1):
            if year > 1:
                NOI *= (1 + NOI성장률)
            이자 = 대출잔액 * r
            원금 = 연간상환액 - 이자
            대출잔액 -= 원금
            과세소득 = NOI - 이자
            세금 = 과세소득 * 법인세율
            세후이익 = 과세소득 - 세금
            연간현금 = 세후이익 - 원금
            세후이익목록.append(세후이익)
            현금흐름.append(연간현금)

        # Exit 계산
        매각가 = NOI / ExitCapRate
        매각차익 = 매각가 - 대출잔액
        현금흐름[-1] += 매각차익

        # IRR 계산
        IRR = npf.irr(현금흐름)

        # 결과 저장
        결과.append({
            "LTV": f"{int(LTV*100)}%",
            "Cap Rate": f"{int(CapRate*100)}%",
            "자기자본(엔)": int(자기자본),
            "Exit 수익(엔)": int(매각차익),
            "IRR": f"{IRR*100:.2f}%",
            "세후 연평균 배당률": f"{(np.mean(세후이익목록)/자기자본)*100:.2f}%"
        })

# ===== DataFrame 변환 및 출력 =====
df = pd.DataFrame(결과)
print(df)

   LTV Cap Rate    자기자본(엔)  Exit 수익(엔)     IRR 세후 연평균 배당률
0  60%       3%  128000000   374669079  10.68%      4.23%
1  60%       4%  128000000   350810370  11.23%      6.24%
2  70%       3%   96000000   353608442  12.22%      5.24%
3  70%       4%   96000000   329749733  12.94%      7.92%
